# $k_{diff}$ vs $F_f$ graph



In [1]:
import importlib
import numpy as np
import sys
sys.path.append('/home/hcleroy/PostDoc/aging_condensates/Simulation/Parallel_Simulation')

import Parallel_Run
importlib.reload(Parallel_Run)

Ec = lambda L,N : 3/2*np.log(L/N* np.pi/3)
import cProfile
import re

# data

In [2]:
media = "/media/hcleroy/Simulation_Res/Parallel_Simulation_aging/make_graph/N3/"
import numpy as np
from Cluster import Cluster
from ISF import ISF
from MSD import MSD
from Energy import NRG
from PCF import PCF
from PCF import PCF_L
from Time import Time

# gillespie parameter
Nlinker = 3
ell_tot = 10**3
kdiff = 1./ell_tot
Energy = -15

Nprocess = 100
seeds = set()
while len(seeds) < Nprocess:
    seeds.add(np.random.randint(1000000))
seeds = list(seeds)
args = [[ell_tot,Energy,kdiff,seeds[_],Nlinker,3] for _ in range(Nprocess)]

# argument of the different classes
cluster_arg = tuple([3.]) # max distance
MSD_arg = () # no argument 
ISF_arg = (0.5,10) # q_norm, q_num_sample
NRG_arg = ()
PCF_arg = (np.sqrt(ell_tot)/2,50) # max_distance,numb_bin
PCF_L_arg = (ell_tot,30) # max_distance,numb_bin
Time_arg = ()

measurement_args = {
    'cluster': (Cluster, cluster_arg),
    'MSD': (MSD, MSD_arg),
    'ISF': (ISF, ISF_arg),
    'PCF':(PCF,PCF_arg),
    'PCF_L':(PCF_L,PCF_L_arg),
    'NRG':(NRG,NRG_arg)#,
    #'Time':(Time,Time_arg)
    # Add other measurements as needed
}

measurement_flags = {
    'NRG':True,
    'cluster': False,
    'MSD': False,
    'ISF': False,
    'PCF':False,
    'PCF_L':False#,
    #'Time':True
    # Set each measurement to True/False as desired
}

# Simulation parameters
step_tot = 1*10**5
#check_steps = 10**2
initial_check_steps = 10**4
coarse_grained_step = 10**1
log_base=1.5

#energies = [-15,-15,-15,-15,-10,-10,-10,-20]
#kdiffs = [1/ell_tot,0.1/ell_tot,0.01/ell_tot,0.001/ell_tot,1/ell_tot,0.1/ell_tot,0.01/ell_tot,1/ell_tot]
#step_tots = [10**5,10**5,2*10**5,5*10**5,10**5,10**5,2*10**5,10**5]
#names = ['ksL_1','ksL_01','ksL_001','ksL_0001','ksL_1E10','ksL_01E10','ksL_001E10','ksL_1E20']

In [8]:
energies = [-15,-15,-20,-20,-100,-100]
energies = [-100,-100]
kdiffs = [1/ell_tot,0.1/ell_tot,1/ell_tot,0.1/ell_tot,1/ell_tot,0.1/ell_tot]
step_tots = [10**5 for _ in range(6)]
names = ['1_100','01_100','1_20','01_20','1_30','01_30']

In [5]:
names.__len__()

6

In [9]:
for n in [0,1]:
    step_tot = step_tots[n]
    Energy = energies[n]
    kdiff= kdiffs[n]
#kdiff = 1./10**3
#step_tot = 10**5
#Energy = -25
#name = 'ksL_1E25'
    args = [[ell_tot,Energy,kdiff,seeds[_],Nlinker,3] for _ in range(Nprocess)]
    Parallel_Run.parallel_evolution(args,step_tot,initial_check_steps,coarse_grained_step,media+names[n]+'.hdf',
                                    measurement_args,measurement_flags,log_base)